In [ ]:
import pyspark
from delta import *
import datetime
from pyspark.sql import SparkSession
import time
import numpy as np
from datetime import datetime, timedelta
from pyspark import StorageLevel
import random
import time
from pyspark.sql.functions import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
            .config("spark.driver.memory", "6g") \ 
            .config("spark.memory.fraction", "0.8") \
            .config("spark.memory.storageFraction", "0.5") \


spark = configure_spark_with_delta_pip(builder, extra_packages=my_packages).getOrCreate()

# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.setLogLevel.html
#spark.sparkContext.setLogLevel("DEBUG") # or TRACE, useful if there are connection issues!

spark

In [ ]:
# Define file paths
data_path = "path_to_tpch_dataset"

# Load TPC-H data into Spark DataFrame
df_customers2 = spark.read.option("delimiter", "|").csv(data_path + "/customer.tbl").toDF(
    "c_custkey", "c_name", "c_address", "c_nationkey", "c_phone", "c_acctbal", "c_mktsegment", "c_comment", "c_dummy"
)
df_lineitem2 = spark.read.option("delimiter", "|").csv(data_path + "/lineitem.tbl").toDF(
    "l_orderkey", "l_partkey", "l_suppkey", "l_linenumber", "l_quantity", "l_extendedprice", 
    "l_discount", "l_tax", "l_returnflag", "l_linestatus", "l_shipdate", "l_commitdate", 
    "l_receiptdate", "l_shipinstruct", "l_shipmode", "l_comment", "l_dummy"
)
df_part2 = spark.read.option("delimiter", "|").csv(data_path + "/part.tbl").toDF(
    "p_partkey", "p_name", "p_mfgr", "p_brand", "p_type", "p_size", "p_container", "p_retailprice", "p_comment", "p_dummy"
)
df_region2 = spark.read.option("delimiter", "|").csv(data_path + "/region.tbl").toDF(
    "r_regionkey", "r_name", "r_comment", "r_dummy"
)
df_supplier2 = spark.read.option("delimiter", "|").csv(data_path + "/supplier.tbl").toDF(
    "s_suppkey", "s_name", "s_address", "s_nationkey", "s_phone", "s_acctbal", "s_comment", "s_dummy"
)
df_partsupp2 = spark.read.option("delimiter", "|").csv(data_path + "/partsupp.tbl").toDF(
    "ps_partkey", "ps_suppkey", "ps_availqty", "ps_supplycost", "ps_comment", "ps_dummy"
)
df_orders2 = spark.read.option("delimiter", "|").csv(data_path + "/orders.tbl").toDF(
    "o_orderkey", "o_custkey", "o_orderstatus", "o_totalprice", "o_orderdate", "o_orderpriority", "o_clerk", "o_shippriority", "o_comment", "o_dummy"
)
df_nation2 = spark.read.option("delimiter", "|").csv(data_path + "/nation.tbl").toDF(
    "n_nationkey", "n_name", "n_regionkey", "n_comment", "n_dummy"
)

In [ ]:
# Create Delta Lake tables ON HDFS
df_customers2.write.format("delta").mode("overwrite").saveAsTable("customer")
df_lineitem2.write.format("delta").mode("overwrite").saveAsTable("lineitem")
df_part2.write.format("delta").mode("overwrite").saveAsTable("part")
df_region2.write.format("delta").mode("overwrite").saveAsTable("region")
df_supplier2.write.format("delta").mode("overwrite").saveAsTable("supplier")
df_partsupp2.write.format("delta").mode("overwrite").saveAsTable("partsupp")
df_orders2.write.format("delta").mode("overwrite").saveAsTable("orders")
df_nation2.write.format("delta").mode("overwrite").saveAsTable("nation")

In [ ]:
#LOAD FROM HDFS

df_nation = spark.read.format("delta").load("path_to_table_nation")
df_orders = spark.read.format("delta").load("path_to_table_orders")
df_partsupp = spark.read.format("delta").load("path_to_table_partsupp")
df_supplier = spark.read.format("delta").load("path_to_table_supplier")
df_region = spark.read.format("delta").load("path_to_table_region")
df_part = spark.read.format("delta").load("path_to_table_part")
df_lineitem = spark.read.format("delta").load("path_to_table_lineitem")
df_customer = spark.read.format("delta").load("path_to_table_customer")
print("done loading")

In [ ]:
# Change storage level according to the different options available on Spark.StorageLevel

df_nation.persist(StorageLevel.DISK_ONLY)
df_orders.persist(StorageLevel.DISK_ONLY)
df_partsupp.persist(StorageLevel.DISK_ONLY)
df_supplier.persist(StorageLevel.DISK_ONLY)
df_region.persist(StorageLevel.DISK_ONLY)
df_part.persist(StorageLevel.DISK_ONLY)
df_lineitem.persist(StorageLevel.DISK_ONLY)
df_customer.persist(StorageLevel.DISK_ONLY)

In [ ]:
# Perform an action to activate persistence, since its a lazy action
df_nation.count()
df_orders.count()
df_partsupp.count()
df_supplier.count()
df_region.count()
df_part.count()
df_lineitem.count()
df_customer.count()

In [ ]:
# Register Delta tables 
df_nation.createOrReplaceTempView("nation")
df_orders.createOrReplaceTempView("orders")
df_partsupp.createOrReplaceTempView("partsupp")
df_supplier.createOrReplaceTempView("supplier")
df_region.createOrReplaceTempView("region")
df_part.createOrReplaceTempView("part")
df_lineitem.createOrReplaceTempView("lineitem")
df_customer.createOrReplaceTempView("customers")

In [ ]:
def q14(spark):
    # Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1998-12-01', '%Y-%m-%d')

    # Generate random dates within the specified range
    date1 = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    date2 = date1 + timedelta(days=random.randint(1, (end_date - date1).days))
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    SELECT
        100.00 * SUM(CASE
            WHEN p_type LIKE 'PROMO%' THEN l_extendedprice * (1 - l_discount)
            ELSE 0
        END) / SUM(l_extendedprice * (1 - l_discount)) AS promo_revenue
    FROM
        lineitem,
        part
    WHERE
        l_partkey = p_partkey
        AND l_shipdate >= DATE '{date1.strftime('%Y-%m-%d')}'
        AND l_shipdate < DATE '{date2.strftime('%Y-%m-%d')}'
    """
    
      
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
     # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
    #result.show()
 
    return query_time

# Example usage:
# spark = SparkSession.builder.appName("PromoRevenueQuery").getOrCreate()
# execution_time = get_promo_revenue_query_time(spark)
# print(f"Query execution time: {execution_time} seconds")


In [ ]:
def q2(spark):

# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')

    
   # Generate random parameters for the query
    p_size = random.randint(1, 50)
    p_type = random.choice(['SMALL', 'MEDIUM', 'LARGE','STANDARD','ECONOMY','PROM','ANODIZED', 'BURNISHED', 'PLATED', 'POLISHED', 'BRUSHED', 'TIN', 'NICKEL', 'BRASS', 'STEEL', 'COPPER'])  # Adjust as needed
    region_name = random.choice(['AFRICA', 'AMERICA', 'ASIA', 'EUROPE', 'MIDDLE EAST'])  # Adjust as needed
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        s_acctbal,
        s_name,
        n_name,
        p_partkey,
        p_mfgr,
        s_address,
        s_phone,
        s_comment
    from
        part,
        supplier,
        partsupp,
        nation,
        region
    where
        p_partkey = ps_partkey
        and s_suppkey = ps_suppkey
        and p_size = {p_size}
        and p_type like '%{p_type}%'
        and s_nationkey = n_nationkey
        and n_regionkey = r_regionkey
        and r_name = '{region_name}'
        and ps_supplycost = (
            select
                min(ps_supplycost)
            from
                partsupp,
                supplier,
                nation,
                region
            where
                p_partkey = ps_partkey
                and s_suppkey = ps_suppkey
                and s_nationkey = n_nationkey
                and n_regionkey = r_regionkey
                and r_name = '{region_name}'
        )
    order by
        s_acctbal desc,
        n_name,
        s_name,
        p_partkey
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
  #  result.show()
    # Calculate the query execution time
    query_time = end_time - start_time

    return query_time

In [ ]:
def q9(spark):

# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')

# Loop through and execute the query multiple times

    # Generate random parameter for the query
    p_name = random.choice(["almond", "antique", "aquamarine", "azure", "beige", "bisque", "black", "blanched", "blue", 
"blush", "brown", "burlywood", "burnished", "chartreuse", "chiffon", "chocolate", "coral", 
"cornflower", "cornsilk", "cream", "cyan", "dark", "deep", "dim", "dodger", "drab", "firebrick", 
"floral", "forest", "frosted", "gainsboro", "ghost", "goldenrod", "green", "grey", "honeydew", 
"hot", "indian", "ivory", "khaki", "lace", "lavender", "lawn", "lemon", "light", "lime", "linen", 
"magenta", "maroon", "medium", "metallic", "midnight", "mint", "misty", "moccasin", "navajo", 
"navy", "olive", "orange", "orchid", "pale", "papaya", "peach", "peru", "pink", "plum", "powder", 
"puff", "purple", "red", "rose", "rosy", "royal", "saddle", "salmon", "sandy", "seashell", "sienna", 
"sky", "slate", "smoke", "snow", "spring", "steel", "tan", "thistle", "tomato", "turquoise", "violet", 
"wheat", "white", "yellow"])  # Adjust as needed
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        nation,
        o_year,
        sum(amount) as sum_profit
    from
        (
            select
                n_name as nation,
                extract(year from o_orderdate) as o_year,
                l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
            from
                part,
                supplier,
                lineitem,
                partsupp,
                orders,
                nation
            where
                s_suppkey = l_suppkey
                and ps_suppkey = l_suppkey
                and ps_partkey = l_partkey
                and p_partkey = l_partkey
                and o_orderkey = l_orderkey
                and s_nationkey = n_nationkey
                and p_name like '%{p_name}%'
        ) as profit
    group by
        nation,
        o_year
    order by
        nation,
        o_year desc
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time

In [ ]:
def q20(spark):


# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')


    # Generate random parameters for the query
    date1 = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
   # date2 = date1 + timedelta(days=random.randint(1, (end_date - date1).days))
    p_name_prefix = random.choice(['a', 'b', 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','y','w','x','z'])  # Adjust as needed
   # ship_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        s_name,
        s_address
    from
        supplier,
        nation
    where
        s_suppkey in (
            select
                ps_suppkey
            from
                partsupp
            where
                ps_partkey in (
                    select
                        p_partkey
                    from
                        part
                    where
                        p_name like '{p_name_prefix}%'
                )
                and ps_availqty > (
                    select
                        0.5 * sum(l_quantity)
                    from
                        lineitem
                    where
                        l_partkey = ps_partkey
                        and l_suppkey = ps_suppkey
                        and l_shipdate >= date '{date1.strftime('%Y-%m-%d')}'
                        and l_shipdate < date '{date1.strftime('%Y-%m-%d')}' + interval '1' year
                )
        )
        and s_nationkey = n_nationkey
        and n_name = 'GERMANY'
    order by
        s_name
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
  #  result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time

In [ ]:
def q6(spark):



# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')


    # Generate random parameters for the query
    ship_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    discount = random.uniform(0.00, 0.10)
    quantity = random.randint(1, 50)
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        sum(l_extendedprice * l_discount) as revenue
    from
        lineitem
    where
        l_shipdate >= date '{ship_date.strftime('%Y-%m-%d')}'
        and l_shipdate < date '{ship_date.strftime('%Y-%m-%d')}' + interval '1' year
        and l_discount between {discount - 0.01} and {discount + 0.01}
        and l_quantity < {quantity}
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
  #  result.show()
    return query_time

In [ ]:
def q17(spark):

# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')

    syllables_1 = ['SM', 'LG', 'MED', 'JUMBO', 'WRAP']
    syllables_2 = ['CASE', 'BOX', 'BAG', 'JAR', 'PKG', 'PACK', 'CAN', 'DRUM']


    # Generate random parameters for the query
    mfgr_num = random.randint(1, 5)
    brand_num = random.randint(1, 5)
   # mfgr = f'Manufacturer{mfgr_num}'
    brand = f'Brand#{brand_num}{mfgr_num}'
    container = f'{random.choice(syllables_1)} {random.choice(syllables_2)}'
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        sum(l_extendedprice) / 7.0 as avg_yearly
    from
        lineitem,
        part
    where
        p_partkey = l_partkey
        and p_brand = '{brand}'
        and p_container = '{container}'
        and l_quantity < (
            select
                0.2 * avg(l_quantity)
            from
                lineitem
            where
                l_partkey = p_partkey
        )
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time


In [ ]:
def q18(spark):

# Define the date range
    start_date = datetime.strptime('1992-01-02', '%Y-%m-%d')
    end_date = datetime.strptime('1992-01-02', '%Y-%m-%d')


    # Generate random parameter for the query
    min_quantity = random.randint(1, 50)  # Adjust as needed
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        c_name,
        c_custkey,
        o_orderkey,
        o_orderdate,
        o_totalprice,
        sum(l_quantity)
    from
        customers,
        orders,
        lineitem
    where
        o_orderkey in (
            select
                l_orderkey
            from
                lineitem
            group by
                l_orderkey having
                    sum(l_quantity) > {min_quantity}
        )
        and c_custkey = o_custkey
        and o_orderkey = l_orderkey
    group by
        c_name,
        c_custkey,
        o_orderkey,
        o_orderdate,
        o_totalprice
    order by
        o_totalprice desc,
        o_orderdate
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time 

In [ ]:
def q8(spark):
    syllables_1 = ['STANDARD', 'SMALL', 'MEDIUM', 'LARGE', 'ECONOMY','PROM']
    syllables_2 = ['ANODIZED', 'BURNISHED', 'PLATED', 'POLISHED', 'BRUSHED']
    syllables_3 = ['TIN', 'NICKEL', 'BRASS', 'STEEL', 'COPPER']
    nations = [
    "ALGERIA", "ARGENTINA", "BRAZIL", "CANADA", "EGYPT",
    "ETHIOPIA", "FRANCE", "GERMANY", "INDIA", "INDONESIA",
    "IRAN", "IRAQ", "JAPAN", "JORDAN", "KENYA", "MOROCCO",
    "MOZAMBIQUE", "PERU", "CHINA", "ROMANIA", "SAUDI ARABIA",
    "VIETNAM", "RUSSIA", "UNITED KINGDOM", "UNITED STATES"
]

    regions = ["AFRICA", "AMERICA", "ASIA", "EUROPE", "MIDDLE EAST"]



    # Generate random parameters for the query
    nation = random.choice(nations)
    region = random.choice(regions)
    part_type = f'{random.choice(syllables_1)} {random.choice(syllables_2)} {random.choice(syllables_3)}'
    
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        o_year,
        sum(case
            when nation = '{nation}' then volume
            else 0
        end) / sum(volume) as mkt_share
    from
        (
            select
                extract(year from o_orderdate) as o_year,
                l_extendedprice * (1 - l_discount) as volume,
                n2.n_name as nation
            from
                part,
                supplier,
                lineitem,
                orders,
                customers,
                nation n1,
                nation n2,
                region
            where
                p_partkey = l_partkey
                and s_suppkey = l_suppkey
                and l_orderkey = o_orderkey
                and o_custkey = c_custkey
                and c_nationkey = n1.n_nationkey
                and n1.n_regionkey = r_regionkey
                and r_name = '{region}'
                and s_nationkey = n2.n_nationkey
                and o_orderdate between date '1995-01-01' and date '1996-12-31'
                and p_type = '{part_type}'
        ) as all_nations
    group by
        o_year
    order by
        o_year
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time

In [ ]:
def q21(spark):

# Define the list of nations
    nations = [
    "ALGERIA", "ARGENTINA", "BRAZIL", "CANADA", "EGYPT",
    "ETHIOPIA", "FRANCE", "GERMANY", "INDIA", "INDONESIA",
    "IRAN", "IRAQ", "JAPAN", "JORDAN", "KENYA", "MOROCCO",
    "MOZAMBIQUE", "PERU", "CHINA", "ROMANIA", "SAUDI ARABIA",
    "VIETNAM", "RUSSIA", "UNITED KINGDOM", "UNITED STATES"]


    # Generate random parameters for the query
    nation = random.choice(nations)
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        s_name,
        count(*) as numwait
    from
        supplier,
        lineitem l1,
        orders,
        nation
    where
        s_suppkey = l1.l_suppkey
        and o_orderkey = l1.l_orderkey
        and o_orderstatus = 'F'
        and l1.l_receiptdate > l1.l_commitdate
        and exists (
            select
                *
            from
                lineitem l2
            where
                l2.l_orderkey = l1.l_orderkey
                and l2.l_suppkey <> l1.l_suppkey
        )
        and not exists (
            select
                *
            from
                lineitem l3
            where
                l3.l_orderkey = l1.l_orderkey
                and l3.l_suppkey <> l1.l_suppkey
                and l3.l_receiptdate > l3.l_commitdate
        )
        and s_nationkey = n_nationkey
        and n_name = '{nation}'
    group by
        s_name
    order by
        numwait desc,
        s_name
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time

In [ ]:
def q13(spark):
    # Generate random parameters for the query
    comment1 = random.choice(['Customer', 'Customer', 'Customer'])
    comment2 = random.choice(['Complaints', 'Recommends'])
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        c_count,
        count(*) as custdist
    from
        (
            select
                c_custkey,
                count(o_orderkey)
            from
                customers left outer join orders on
                    c_custkey = o_custkey
                    and o_comment not like '%{comment1}%{comment2}%'
            group by
                c_custkey
        ) as c_orders (c_custkey, c_count)
    group by
        c_count
    order by
        custdist desc,
        c_count desc
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time


In [ ]:
def q3(spark):
    # Generate random parameters for the query
    segment = random.choice(["AUTOMOBILE", "BUILDING", "FURNITURE", "HOUSEHOLD", "MACHINERY"])
     # Generate two random dates
    start_date = datetime(1992, 1, 2)  # Start date
    end_date = datetime(1992, 1, 30)    # End date
    date1 = start_date + timedelta(days=random.randint(0, (end_date - start_date - timedelta(days=1)).days))
    date2 = date1 + timedelta(days=random.randint(1, (end_date - date1).days))  # Ensure date2 > date1
    
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        l_orderkey,
        sum(l_extendedprice * (1 - l_discount)) as revenue,
        o_orderdate,
        o_shippriority
    from
        customers,
        orders,
        lineitem
    where
        c_mktsegment = '{segment}'
        and c_custkey = o_custkey
        and l_orderkey = o_orderkey
        and o_orderdate < date '{date1}'
        and l_shipdate > date '{date2}'
    group by
        l_orderkey,
        o_orderdate,
        o_shippriority
    order by
        revenue desc,
        o_orderdate
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time


In [ ]:
def q22(spark):
    country_codes = ["AL", "AR", "BR", "CA", "EG", "ET", "FR", "DE", "IN", "ID", 
                 "IR", "IQ", "JP", "JO", "KE", "MA", "MZ", "PE", "CN", "RO", 
                 "SA", "VN", "RU", "UK", "US"]

# Concatenate the parts to form the phone number
    #country_codes_str = f"{country_codes[country_code_index]}-{local_number1}-{local_number2}-{local_number3}"

    country_codes = [str(random.randint(1, len(country_codes) - 1)).zfill(2) for _ in range(7)]  # Generate 7 random country codes
    country_codes_str = "', '".join(country_codes)
    
    # Start the timer
    start_time = time.time()
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        cntrycode,
        count(*) as numcust,
        sum(c_acctbal) as totacctbal
    from
        (
            select
                substring(c_phone from 1 for 2) as cntrycode,
                c_acctbal
            from
                customers
            where
                substring(c_phone from 1 for 2) in
                    ('{country_codes_str}')
                and c_acctbal > (
                    select
                        avg(c_acctbal)
                    from
                        customers
                    where
                        c_acctbal > 0.00
                        and substring(c_phone from 1 for 2) in
                            ('{country_codes_str}')
                )
                and not exists (
                    select
                        *
                    from
                        orders
                    where
                        o_custkey = c_custkey
                )
        ) as custsale
    group by
        cntrycode
    order by
        cntrycode
    """
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time


In [ ]:
def q16(spark):
    # Generate random parameters for the query
    mfgr_num = random.randint(1, 5)
    brand_num = random.randint(1, 5)
   # mfgr = f'Manufacturer{mfgr_num}'
    brand = f'Brand#{brand_num}{mfgr_num}'
    type_prefix = random.choice(['S', 'M', 'L','E','A', 'B', 'P','T', 'N', 'C'])  # Adjust as needed
    
    size_list = random.sample(range(1, 50), 8)
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        p_brand,
        p_type,
        p_size,
        count(distinct ps_suppkey) as supplier_cnt
    from
        partsupp,
        part
    where
        p_partkey = ps_partkey
        and p_brand <> '{brand}'
        and not p_type like '{type_prefix}%'
        and p_size in ({', '.join(str(size) for size in size_list)})
        and ps_suppkey not in (
            select
                s_suppkey
            from
                supplier
            where
                s_comment like '%Customer%Complaints%'
        )
    group by
        p_brand,
        p_type,
        p_size
    order by
        supplier_cnt desc,
        p_brand,
        p_type,
        p_size
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time


In [ ]:
def q4(spark):

# Define start and end dates
    start_date = datetime(1992, 1, 1)
    end_date = datetime(1998, 12, 31)


   # Generate random start date within the defined range
    random_start_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        o_orderpriority,
        count(*) as order_count
    from
        orders
    where
        o_orderdate >= date '{random_start_date.strftime("%Y-%m-%d")}'
        and o_orderdate < date '{random_start_date.strftime("%Y-%m-%d")}' + interval '3' month
        and exists (
            select
                *
            from
                lineitem
            where
                l_orderkey = o_orderkey
                and l_commitdate < l_receiptdate
        )
    group by
        o_orderpriority
    order by
        o_orderpriority
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time

In [ ]:
def q11(spark):
    # Generate random values for parameters
    nation_name = random.choice(["ALGERIA", "ARGENTINA", "BRAZIL", "CANADA", "EGYPT", "ETHIOPIA", "FRANCE", "GERMANY", "INDIA", "INDONESIA", "IRAN", "IRAQ", "JAPAN", "JORDAN", "KENYA", "MOROCCO", "MOZAMBIQUE", "PERU", "CHINA", "ROMANIA", "SAUDI ARABIA", "VIETNAM", "RUSSIA", "UNITED KINGDOM", "UNITED STATES"])
    threshold = (0.0001 / 1)  # 0,0001 / SF
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        ps_partkey,
        sum(ps_supplycost * ps_availqty) as value
    from
        partsupp,
        supplier,
        nation
    where
        ps_suppkey = s_suppkey
        and s_nationkey = n_nationkey
        and n_name = '{nation_name}'
    group by
        ps_partkey
    having
        sum(ps_supplycost * ps_availqty) > (
            select
                sum(ps_supplycost * ps_availqty) * {threshold}
            from
                partsupp,
                supplier,
                nation
            where
                ps_suppkey = s_suppkey
                and s_nationkey = n_nationkey
                and n_name = '{nation_name}'
        )
    order by
        value desc
    """

    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
  #  result.show()
    return query_time

In [ ]:
def q15(spark):
    start_date = datetime(1992, 1, 2)  # Start date
    end_date = datetime(1998, 12, 31)    # End date


    # Generate random start date
    random_start_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    # Translate SQL Query into Spark SQL syntax for creating the view
    create_view_query = f"""
    CREATE OR REPLACE TEMPORARY VIEW revenue_s AS
    SELECT
        l_suppkey AS supplier_no,
        SUM(l_extendedprice * (1 - l_discount)) AS total_revenue
    FROM
        lineitem
    WHERE
        l_shipdate >= DATE '{random_start_date.strftime("%Y-%m-%d")}'
        AND l_shipdate < DATE '{random_start_date.strftime("%Y-%m-%d")}' + INTERVAL '3' MONTH
    GROUP BY
        l_suppkey
    """
    # Start the timer
    start_time = time.time()
    # Execute the create view query
    spark.sql(create_view_query)
    
    # Translate SQL Query into Spark SQL syntax for selecting from the view
    select_query = """
    SELECT
        s_suppkey,
        s_name,
        s_address,
        s_phone,
        total_revenue
    FROM
        supplier
    JOIN
        revenue_s
    ON
        s_suppkey = supplier_no
    AND
        total_revenue = (
            SELECT
                MAX(total_revenue)
            FROM
                revenue_s
        )
    ORDER BY
        s_suppkey
    """

    # Execute the query
    result = spark.sql(select_query)
    
    # Stop the timer
    end_time = time.time()
    

    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time

In [ ]:
def q1(spark):
    # Randomize the parameter value
    param_value = random.randint(1, 30)  # Random value between 1 and 30
    
    # Start the timer
    start_time = time.time()
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    SELECT
        l_returnflag,
        l_linestatus,
        SUM(l_quantity) AS sum_qty,
        SUM(l_extendedprice) AS sum_base_price,
        SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
        SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
        AVG(l_quantity) AS avg_qty,
        AVG(l_extendedprice) AS avg_price,
        AVG(l_discount) AS avg_disc,
        COUNT(*) AS count_order
    FROM
        lineitem
    WHERE
        l_shipdate <= DATE '1998-12-01' - INTERVAL '{param_value}' DAY
    GROUP BY
        l_returnflag,
        l_linestatus
    ORDER BY
        l_returnflag,
        l_linestatus
    """
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()  
    return query_time

In [ ]:
def q10(spark):
    start_date = datetime(1992, 1, 2)  # Start date
    end_date = datetime(1998, 12, 31)    # End date


    #random date
    random_start_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    SELECT
        c_custkey,
        c_name,
        SUM(l_extendedprice * (1 - l_discount)) AS revenue,
        c_acctbal,
        n_name,
        c_address,
        c_phone,
        c_comment
    FROM
        customers,
        orders,
        lineitem,
        nation
    WHERE
        c_custkey = o_custkey
        AND l_orderkey = o_orderkey
        AND o_orderdate >= DATE '{random_start_date.strftime("%Y-%m-%d")}'
        AND o_orderdate < DATE '{random_start_date.strftime("%Y-%m-%d")}' + INTERVAL '3' MONTH
        AND l_returnflag = 'R'
        AND c_nationkey = n_nationkey
    GROUP BY
        c_custkey,
        c_name,
        c_acctbal,
        c_phone,
        n_name,
        c_address,
        c_comment
    ORDER BY
        revenue DESC
    """
     # Start the timer
    start_time = time.time()
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time


In [ ]:
def q19(spark):
    # Generate random parameter values
    p_brand_values = [f'Brand#{random.randint(1, 5)}{random.randint(1, 5)}', f'Brand#{random.randint(1, 5)}{random.randint(1, 5)}', f'Brand#{random.randint(1, 5)}{random.randint(1, 5)}']
    l_quantity_values = [random.randint(1, 50), random.randint(1, 50), random.randint(1, 50)]
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        sum(l_extendedprice * (1 - l_discount)) as revenue
    from
        lineitem,
        part
    where
        (
            p_partkey = l_partkey
            and p_brand = '{p_brand_values[0]}'
            and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
            and l_quantity >= {l_quantity_values[0]} and l_quantity <= {l_quantity_values[0]} + 10
            and p_size between 1 and 5
            and l_shipmode in ('AIR', 'AIR REG')
            and l_shipinstruct = 'DELIVER IN PERSON'
        )
        or
        (
            p_partkey = l_partkey
            and p_brand = '{p_brand_values[1]}'
            and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
            and l_quantity >= {l_quantity_values[1]} and l_quantity <= {l_quantity_values[1]} + 10
            and p_size between 1 and 10
            and l_shipmode in ('AIR', 'AIR REG')
            and l_shipinstruct = 'DELIVER IN PERSON'
        )
        or
        (
            p_partkey = l_partkey
            and p_brand = '{p_brand_values[2]}'
            and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
            and l_quantity >= {l_quantity_values[2]} and l_quantity <= {l_quantity_values[2]} + 10
            and p_size between 1 and 15
            and l_shipmode in ('AIR', 'AIR REG')
            and l_shipinstruct = 'DELIVER IN PERSON'
        )
    """

    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
   # result.show()
    # Calculate the query execution time
    query_time = end_time - start_time
    
    return query_time

In [ ]:
def q5(spark):
# Define start and end dates
    start_date = datetime(1992, 1, 1)
    end_date = datetime(1998, 12, 31)


   # Generate random start date within the defined range
    random_start_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    region_name = random.choice(['AFRICA', 'AMERICA', 'ASIA', 'EUROPE', 'MIDDLE EAST']) 
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        n_name,
        sum(l_extendedprice * (1 - l_discount)) as revenue
    from
        customers,
        orders,
        lineitem,
        supplier,
        nation,
        region
    where
        c_custkey = o_custkey
        and l_orderkey = o_orderkey
        and l_suppkey = s_suppkey
        and c_nationkey = s_nationkey
        and s_nationkey = n_nationkey
        and n_regionkey = r_regionkey
        and r_name = '{region_name}'  
        and o_orderdate >= date '{random_start_date.strftime("%Y-%m-%d")}'
        and o_orderdate < date '{random_start_date.strftime("%Y-%m-%d")}' + interval '1' year
    group by
        n_name
    order by
        revenue desc
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time

In [ ]:
def q7(spark):

# Define the nations
    nations = ["ALGERIA", "ARGENTINA", "BRAZIL", "CANADA", "EGYPT", "ETHIOPIA", "FRANCE", "GERMANY", "INDIA", "INDONESIA", 
           "IRAN", "IRAQ", "JAPAN", "JORDAN", "KENYA", "MOROCCO", "MOZAMBIQUE", "PERU", "CHINA", "ROMANIA", 
           "SAUDI ARABIA", "VIETNAM", "RUSSIA", "UNITED KINGDOM", "UNITED STATES"]


    # Select random nations
    supp_nation = random.choice(nations)
    cust_nation = random.choice(nations)
    
    # Ensure supp_nation and cust_nation are different
    while supp_nation == cust_nation:
        cust_nation = random.choice(nations)
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        supp_nation,
        cust_nation,
        l_year,
        sum(volume) as revenue
    from
        (
            select
                n1.n_name as supp_nation,
                n2.n_name as cust_nation,
                extract(year from l_shipdate) as l_year,
                l_extendedprice * (1 - l_discount) as volume
            from
                supplier,
                lineitem,
                orders,
                customers,
                nation n1,
                nation n2
            where
                s_suppkey = l_suppkey
                and o_orderkey = l_orderkey
                and c_custkey = o_custkey
                and s_nationkey = n1.n_nationkey
                and c_nationkey = n2.n_nationkey
                and (
                    (n1.n_name = '{supp_nation}' and n2.n_name = '{cust_nation}')
                    or (n1.n_name = '{cust_nation}' and n2.n_name = '{supp_nation}')
                )
                and l_shipdate between date '1995-01-01' and date '1996-12-31'
        ) as shipping
    group by
        supp_nation,
        cust_nation,
        l_year
    order by
        supp_nation,
        cust_nation,
        l_year
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    
    
    # Calculate the query execution time
    query_time = end_time - start_time
    
   # result.show()
    
    return query_time

In [ ]:
def q12(spark):
# Define shipping modes
    shipping_modes = ['AIR', 'SHIP', 'RAIL', 'TRUCK', 'MAIL', 'FOB', 'REG AIR']

# Define start and end dates
    start_date = datetime(1992, 1, 1)
    end_date = datetime(1998, 12, 31)


    # Generate random parameters
    random_shipping_mode1, random_shipping_mode2 = random.sample(shipping_modes, 2)
    random_start_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    # Translate SQL Query into Spark SQL syntax
    query = f"""
    select
        l_shipmode,
        sum(case
            when o_orderpriority = '1-URGENT'
                or o_orderpriority = '2-HIGH'
                then 1
            else 0
        end) as high_line_count,
        sum(case
            when o_orderpriority <> '1-URGENT'
                and o_orderpriority <> '2-HIGH'
                then 1
            else 0
        end) as low_line_count
    from
        orders,
        lineitem
    where
        o_orderkey = l_orderkey
        and l_shipmode in ('{random_shipping_mode1}', '{random_shipping_mode2}')
        and l_commitdate < l_receiptdate
        and l_shipdate < l_commitdate
        and l_receiptdate >= date '{random_start_date.strftime("%Y-%m-%d")}'
        and l_receiptdate < date '{random_start_date.strftime("%Y-%m-%d")}' + interval '1' year
    group by
        l_shipmode
    order by
        l_shipmode
    """
    
    # Start the timer
    start_time = time.time()
    
    # Execute the query
    result = spark.sql(query)
    
    # Stop the timer
    end_time = time.time()
    # Calculate the query execution time
    query_time = end_time - start_time
   # result.show()
    return query_time

In [ ]:
a1=0;
a2=0;
a3=0;
a4=0;
a5=0;
a6=0;
a7=0;
a8=0;
a9=0;
a10=0;
a11=0;
a12=0;
a13=0;
a14=0;
a15=0;
a16=0;
a17=0;
a18=0;
a19=0;
a20=0;
a21=0;
a22=0;

In [ ]:
num_queries = 40
# The list to iterate over -> TPC_H throughput test
my_list = [
    21,3,18,5,11,7,6,20,17,12,16,15,13,10,2,8,14,19,9,22,1,4,
    6,17,14,16,19,10,9,2,15,8,5,22, 12,7,13,18,1,4,20,3,11,21,
    8,5,4,6,17,7,1,18,22,14,9,10,15,11,20,2,21,19,13,16,12,3,
    5,21,14,19,15,17,12,6,4,9,8,16,11,2,10,18,1,13,7,22,3,20,
    21,15,4,6,7,16,19,18,14,22,11,13,3,1,2,5,8,20,12,17,10,9,
    10,3,15,13,6,8,9,7,4,11,22,18,12,1,5,16,2,14,19,20,17,21,
    18,8,20,21,2,4,22,17,1,11,9,19,3,13,5,7,10,16,6,14,15,12,
    19,1,15,17,5,8,9,12,14,7,4,3,20,16,6,22,10,13,2,21,18,11,
    8,13,2,20,17,3,6,21,18,11,19,10,15,4,22,1,7,12,9,14,5,16,
    6,15,18,17,12,1,7,2,22,13,21,10,14,9,3,16,20,19,11,4,8,5,
    15,14,18,17,10,20,16,11,1,8,4,22,5,12,3,9,21,2,13,6,19,7,
    1,7,16,17,18,22,12,6,8,9,11,4,2,5,20,21,13,10,19,3,14,15,
    21,17,7,3,1,10,12,22,9,16,6,11,2,4,5,14,8,20,13,18,15,19,
    2,9,5,4,18,1,20,15,16,17,7,21,13,14,19,8,22,11,10,3,12,6,
    16,9,17,8,14,11,10,12,6,21,7,3,15,5,22,20,1,13,19,2,4,18,
    1,3,6,5,2,16,14,22,17,20,4,9,10,11,15,8,12,19,18,13,7,21,
    3,16,5,11,21,9,2,15,10,18,17,7,8,19,14,13,1,4,22,20,6,12,
    14,4,13,5,21,11,8,6,3,17,2,20,1,19,10,9,12,18,15,7,22,16,
    4,12,22,14,5,15,16,2,8,10,17,9,21,7,3,6,13,18,11,20,19,1,
    16,15,14,13,4,22,18,19,7,1,12,17,5,10,20,3,9,21,11,2,6,8,
    20,14,21,12,15,17,4,19,13,10,11,1,16,5,18,7,8,22,9,6,3,2,
    16,14,13,2,21,10,11,4,1,22,18,12,19,5,7,8,6,3,15,20,9,17,
    18,15,9,14,12,2,8,11,22,21,16,1,6,17,5,10,19,4,20,13,3,7,
    7,3,10,14,13,21,18,6,20,4,9,8,22,15,2,1,5,12,19,17,11,16,
    18,1,13,7,16,10,14,2,19,5,21,11,22,15,8,17,20,3,4,12,6,9,
    13,2,22,5,11,21,20,14,7,10,4,9,19,18,6,3,1,8,15,12,17,16,
    14,17,21,8,2,9,6,4,5,13,22,7,15,3,1,18,16,11,10,12,20,19,
    10,22,1,12,13,18,21,20,2,14,16,7,15,3,4,17,5,19,6,8,9,11,
    10,8,9,18,12,6,1,5,20,11,17,22,16,3,13,2,15,21,14,19,7,4,
    7,17,22,5,3,10,13,18,9,1,14,15,21,19,16,12,8,6,11,20,4,2,
    2,9,21,3,4,7,1,11,16,5,20,19,18,8,17,13,10,12,15,6,14,22,
    15,12,8,4,22,13,16,17,18,3,7,5,6,1,9,11,21,10,14,20,19,2,
    15,16,2,11,17,7,5,14,20,4,21,3,10,9,12,8,13,6,18,19,22,1,
    1,13,11,3,4,21,6,14,15,22,18,9,7,5,10,20,12,16,17,8,19,2,
    14,17,22,20,8,16,5,10,1,13,2,21,12,9,4,18,3,7,6,19,15,11,
    9,17,7,4,5,13,21,18,11,3,22,1,6,16,20,14,15,10,8,2,12,19,
    13,14,5,22,19,11,9,6,18,15,8,10,7,4,17,16,3,1,12,2,21,20,
    20,5,4,14,11,1,6,16,8,22,7,3,2,12,21,19,17,13,10,15,18,9,
    3,7,14,15,6,5,21,20,18,10,4,16,19,1,13,9,8,17,11,12,22,2,
    13,15,17,1,22,11,3,4,7,20,14,21,9,8,2,18,16,6,10,12,5,19,
    
]

# Iterate over the elements of the list and perform queries based on the value
for element in my_list:
    if element == 1:
        a1 += q1(spark)
    elif element == 2:
        a2 += q2(spark)
    elif element == 3:
        a3 += q3(spark)
    elif element == 4:
        a4 += q4(spark)
    elif element == 5:
        a5 += q5(spark)
    elif element == 6:
        a6 += q6(spark)
    elif element == 7:
        a7 += q7(spark)
    elif element == 8:
        a8 += q8(spark)
    elif element == 9:
        a9 += q9(spark)
    elif element == 10:
        a10 += q10(spark)
    elif element == 11:
        a11 += q11(spark)
    elif element == 12:
        a12 += q12(spark)
    elif element == 13:
        a13 += q13(spark)
    elif element == 14:
        a14 += q14(spark)
    elif element == 15:
        a15 += q15(spark)
    elif element == 16:
        a16 += q16(spark)
    elif element == 17:
        a17 += q17(spark)
    elif element == 18:
        a18 += q18(spark)
    elif element == 19:
        a19 += q19(spark)
    elif element == 20:
        a20 += q20(spark)
    elif element == 21:
        a21 += q21(spark)
    elif element == 22:
        a22 += q22(spark)
        
a1 = a1/num_queries
a2 = a2/num_queries
a3 = a3/num_queries
a4 = a4/num_queries
a5 = a5/num_queries
a6 = a6/num_queries
a7 = a7/num_queries
a8 = a8/num_queries
a9 = a9/num_queries
a10 = a10/num_queries
a11 = a11/num_queries
a12 = a12/num_queries
a13 = a13/num_queries
a14 = a14/num_queries
a15 = a15/num_queries
a16 = a16/num_queries
a17 = a17/num_queries
a18 = a18/num_queries
a19 = a19/num_queries
a20 = a20/num_queries
a21 = a21/num_queries
a22 = a22/num_queries

In [ ]:

print(f"\nAverage query 1 execution time: {a1:.4f} seconds")
print(f"\nAverage query 2 execution time: {a2:.4f} seconds")
print(f"\nAverage query 3 execution time: {a3:.4f} seconds")
print(f"\nAverage query 4 execution time: {a4:.4f} seconds")
print(f"\nAverage query 5 execution time: {a5:.4f} seconds")
print(f"\nAverage query 6 execution time: {a6:.4f} seconds")
print(f"\nAverage query 7 execution time: {a7:.4f} seconds")
print(f"\nAverage query 8 execution time: {a8:.4f} seconds")
print(f"\nAverage query 9 execution time: {a9:.4f} seconds")
print(f"\nAverage query 10 execution time: {a10:.4f} seconds")
print(f"\nAverage query 11 execution time: {a11:.4f} seconds")
print(f"\nAverage query 12 execution time: {a12:.4f} seconds")
print(f"\nAverage query 13 execution time: {a13:.4f} seconds")
print(f"\nAverage query 14 execution time: {a14:.4f} seconds")
print(f"\nAverage query 15 execution time: {a15:.4f} seconds")
print(f"\nAverage query 16 execution time: {a16:.4f} seconds")
print(f"\nAverage query 17 execution time: {a17:.4f} seconds")
print(f"\nAverage query 18 execution time: {a18:.4f} seconds")
print(f"\nAverage query 19 execution time: {a19:.4f} seconds")
print(f"\nAverage query 20 execution time: {a20:.4f} seconds")
print(f"\nAverage query 21 execution time: {a21:.4f} seconds")
print(f"\nAverage query 22 execution time: {a22:.4f} seconds")
aTotal = a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8 + a9 + a10 + a11 + a12 + a13 + a14 + a15 + a16 + a17 + a18 + a19 + a20 + a21 + a22
print(f"\nTotal average time to execute: {aTotal:.4f} seconds")